In [11]:
import pandas as pd
import numpy as np

import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
with open("issues_final.json", 'r') as clean_json_file:
    data = json.load(clean_json_file)
    issues_df=pd.DataFrame(data, columns=['id', 'title', 'body', 'comments', 'originalId', 'originalTitle', 'originalBody'])

    stop_words_l=stopwords.words('english')

    issues_df['title_cleaned']=issues_df.title.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    issues_df['body_cleaned']=issues_df.body.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in str(x).split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    issues_df['original_title_cleaned']=issues_df.originalTitle.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    issues_df['original_body_cleaned']=issues_df.originalBody.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

    print(issues_df.head())

In [37]:
#TFIDF Vectorizer
tfidfvectoriser=TfidfVectorizer()

tfidfvectoriser.fit(issues_df.title_cleaned)

tfidf_vectors=tfidfvectoriser.transform(issues_df.title_cleaned)



#Pairwise
pairwise_similarities = np.dot(tfidf_vectors, tfidf_vectors.T).toarray()
pairwise_differences=euclidean_distances(tfidf_vectors)




In [39]:
def most_similar(issue_id,similarity_matrix,matrix):
    print (f'Issue: {issues_df.iloc[issue_id]["title_cleaned"]}')
    print ('\n')
    print ('Similar Issues:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[issue_id])
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[issue_id])
    for ix in similar_ix:
        if ix >= issue_id:
            continue
        if (matrix == 'Euclidean Distance' and similarity_matrix[issue_id][ix] != 0 and similarity_matrix[issue_id][ix] < 1.1):
          print (f'Issue: {issues_df.iloc[ix]["title_cleaned"]}')
          print (f'{matrix} : {similarity_matrix[issue_id][ix]}')
          print('\n')

        if (matrix == 'Cosine Similarity' and similarity_matrix[issue_id][ix] != 0 and similarity_matrix[issue_id][ix] > 0.35):
          print (f'Issue: {issues_df.iloc[ix]["title_cleaned"]}')
          print (f'{matrix} : {similarity_matrix[issue_id][ix]}')
          print('\n')


In [40]:
for i in range(0, issues_df.shape[0]):
  most_similar(i,pairwise_differences,'Euclidean Distance')  
  most_similar(i,pairwise_similarities,'Cosine Similarity')

Streaming output truncated to the last 5000 lines.


Issue:  fonts  icon request  move down
Cosine Similarity : 0.5362977213835313


Issue:  fonts  icon request  wallet
Cosine Similarity : 1.0


Issue:  fonts  icon request  wallet
Cosine Similarity : 1.0
Issue:  fonts  icon request  wallet


Similar Issues:


Issue:  fonts  icon request  move down
Euclidean Distance : 0.963018461522383


Issue:  fonts  icon request  electric bolt
Euclidean Distance : 0.9704961800449186


Issue:  fonts  icon request  satellite alt
Euclidean Distance : 0.9908752678586373


Issue:  fonts  icon request  lock person
Euclidean Distance : 0.9908752678586373


Issue:  fonts  icon request  solar power
Euclidean Distance : 0.9908752678586373


Issue:  fonts  icon request  network wifi  x  bar
Euclidean Distance : 1.0257590055039125


Issue:  fonts  icon request  expand circle down
Euclidean Distance : 1.0452627338477896
Issue:  fonts  icon request  electric bolt


Similar Issues:


Issue:  fonts  icon request  e